In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#! pip install git+https://github.com/beroguedou/SpecAugment.git
#!pip install torchaudio
#!pip install torchsummary

In [3]:
import os
import csv
import time
import librosa

import warnings
import pandas as pd
from models import *
from utils import *
from decode import *
import torch
import torchaudio
import numpy as np
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer

warnings.filterwarnings('ignore')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed_value = 2020
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)


Bad key "text.kerning_factor" on line 4 in
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [4]:

limit = 10000
params = {'batch_size': None,
          'shuffle': True,
          'num_workers': 10,
          'drop_last': True}


In [7]:
%%time 

training_set = LibriSpeechDataset(limit=limit, n_frames=800, version='train-clean-360')
dev_set = LibriSpeechDataset(limit=limit, n_frames=800, version='dev-clean')


CPU times: user 337 ms, sys: 48.1 ms, total: 385 ms
Wall time: 1.09 s


In [ ]:
# Let's see a single utterance

LibriSpeechDataset(limit=limit, n_frames=800, version='train-clean-360')[0]

In [5]:
units = 256

encoder = EncoderCONV2DRNN(device=device, hidden_size=units).to(device)
decoder = DecoderATTRNN(vocab_size=30000, dec_units=units, hidden_size=units, 
                        encoder_timestamp=encoder.encoder_timestamp).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [8]:
# Train the model

global_trainer(10, training_set, dev_set, params, encoder, decoder, encoder_optimizer,
               decoder_optimizer, criterion, device)

 ====== ====== ====== ====== ====== ======
      The model has 26799272 parameters
 ====== ====== ====== ====== ====== ======


Epoch       10: 100%|████████████████████| 625/625 [36:55<00:00,  3.54s/it, Train loss 5.9240 Eval loss 4.0338]

Time taken for the training 6.1603 hours



In [21]:
mfccs, references = training_set[1]
tokenizer =  BertTokenizer.from_pretrained('bert-base-uncased')
references = [tokenizer.convert_ids_to_tokens(ind) for ind in references.numpy().tolist()]
evaluate(mfccs.unsqueeze(0), references, 40, encoder, decoder, targ_lang_tokenizer=tokenizer, 
          device=device, beam_search=True, beam_width=10)

Input: ['[CLS]', '[CLS]', 'what', 'did', 'you', 'do', 'next', 'i', 'awakened', 'with', 'a', 'sudden', 'start', 'just', 'before', 'six', 'o', "'", 'clock', 'i', 'had', 'not', 'set', 'an', 'alarm', 'though', 'i', 'wanted', 'to', 'get', 'up', 'early', 'to', 'do', 'a', 'little', 'repair', 'job', 'i', 'had']


Predicted translation: ['[CLS]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']


Bleu score: 2.3295942588771784e-155


In [ ]:
# Vérifications

# 1 - Data Augmentation
# 2 - Encoder
# 3 - Attention Mechanism Bahdanau Audio
# 4 - Smoothing and Topk to the attention
# 5 - Decoder 
# 6 - Métrique BLEU

 

In [24]:
!pip freeze > recquirements.txt